# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [2]:
df_raw = pd.read_csv("C:/Users/n0rv1/Desktop/DataScience/ga-data-science/admissions.csv")
df = df_raw.dropna()
df['prestige']=df.prestige.astype(int)
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61         3
1      1  660.0  3.67         3
2      1  800.0  4.00         1
3      1  640.0  3.19         4
4      0  520.0  2.93         4


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [3]:
df.dtypes

admit         int64
gre         float64
gpa         float64
prestige      int32
dtype: object

## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [4]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(index=df['admit'],columns=df['prestige'])

prestige,1,2,3,4
admit,,,,
0,28,95,93,55
1,33,53,28,12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
df_rank = df.join(pd.get_dummies(df['prestige'], drop_first=False, prefix="prestige"))
df_rank.head()

,admit,gre,gpa,prestige,prestige_1,prestige_2,prestige_3,prestige_4
0,0,380.0,3.61,3,0,0,1,0
1,1,660.0,3.67,3,0,0,1,0
2,1,800.0,4.00,1,1,0,0,0
3,1,640.0,3.19,4,0,0,0,1
4,0,520.0,2.93,4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 4

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [6]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df_rank[cols_to_keep].join(df_rank.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [7]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
cross= pd.crosstab(index=df_rank['admit'],columns=df_rank['prestige_1'], margins = True)
print cross

prestige_1    0   1  All
admit                   
0           243  28  271
1            93  33  126
All         336  61  397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [8]:
import statsmodels.formula.api as smf
lm_rank = smf.logit(formula = "admit ~ prestige_1",data = df_rank).fit()

Optimization terminated successfully.
         Current function value: 0.605241
         Iterations 5


In [9]:
print lm_rank.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      395
Method:                           MLE   Df Model:                            1
Date:                Fri, 30 Jun 2017   Pseudo R-squ.:                 0.03143
Time:                        23:17:55   Log-Likelihood:                -240.28
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 7.855e-05
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.9605      0.122     -7.877      0.000        -1.199    -0.721
prestige_1     1.1248      0.284      3.955      0.000         0.567     1.682


In [10]:
#odds Ratio
print np.exp(lm_rank.params)

Intercept     0.382716
prestige_1    3.079493
dtype: float64


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [11]:
#manual Calc
odds_rankothers = cross.ix[1,0]/float(cross.ix[0,0])
print odds_rankothers

0.382716049383


In [12]:
#manual calc
oddsratio_rankothers = np.exp(odds_rankothers)
print oddsratio_rankothers

1.46626162506


In [13]:
import statsmodels.formula.api as smf
lm_notrank = smf.logit(formula = "admit ~ prestige_2 + prestige_3 + prestige_4",data = df_rank).fit()

Optimization terminated successfully.
         Current function value: 0.593375
         Iterations 5


In [14]:
print lm_notrank.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      393
Method:                           MLE   Df Model:                            3
Date:                Fri, 30 Jun 2017   Pseudo R-squ.:                 0.05042
Time:                        23:17:56   Log-Likelihood:                -235.57
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.533e-05
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.1643      0.257      0.639      0.523        -0.339     0.668
prestige_2    -0.7479      0.309     -2.421      0.015        -1.353    -0.142
prestige_3    -1.3647      0.335     -4.069      0.0

In [15]:
print lm_notrank.conf_int()

                   0         1
Intercept  -0.339287  0.667893
prestige_2 -1.353297 -0.142479
prestige_3 -2.022044 -0.707352
prestige_4 -2.488957 -0.884502


#### 3.3 Calculate the odds ratio

In [16]:
print np.exp(lm_notrank.params)

Intercept     1.178571
prestige_2    0.473365
prestige_3    0.255458
prestige_4    0.185124
dtype: float64


#### 3.4 Write this finding in a sentenance: 

Answer: The odd of being admitted decreased by about 53% if the prestige of the school is 2. 

#### 3.5 Print the cross tab for prestige_4

In [17]:
pd.crosstab(df_rank['prestige_4'],df_rank['admit'])

admit,0,1
prestige_4,,
0,216,114
1,55,12


#### 3.6 Calculate the OR 

In [18]:
lm_p4 = smf.logit(formula='admit~prestige_4',data = df_rank).fit()
print np.exp(lm_p4.params)

Optimization terminated successfully.
         Current function value: 0.615129
         Iterations 5
Intercept     0.527778
prestige_4    0.413397
dtype: float64


In [19]:
print np.exp(lm_p4.params)

Intercept     0.527778
prestige_4    0.413397
dtype: float64


#### 3.7 Write this finding in a sentence

Answer: The odd of being admitted decreased by about 58% if the prestige of the school is 4.

## Part 4. Analysis

In [20]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df_rank[cols_to_keep].join(df_rank.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           1           0
1      1  660.0  3.67           0           1           0
2      1  800.0  4.00           0           0           0
3      1  640.0  3.19           0           0           1
4      0  520.0  2.93           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [21]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [22]:
train_cols = data.columns[1:]

#### 4.2 Fit the model

In [23]:
lm_f = smf.logit(formula = 'admit~gre+gpa+prestige_2+prestige_3+prestige_4',data=data).fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [24]:
print lm_f.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Fri, 30 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        23:17:57   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -3.8769      1.142     -3.393      0.001        -6.116    -1.638
gre            0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa            0.7793      0.333      2.344      0.0

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [25]:
#Odds Ratios
print np.exp(lm_f.params)

Intercept     0.020716
gre           1.002221
gpa           2.180027
prestige_2    0.506548
prestige_3    0.262192
prestige_4    0.211525
dtype: float64


In [26]:
params = lm_f.params
conf = lm_f.conf_int()
conf['OR'] = params
conf.columns = ['5%', '95%', 'OR'] #QNS
print np.exp(conf)

                  5%       95%        OR
Intercept   0.002207  0.194440  0.020716
gre         1.000074  1.004372  1.002221
gpa         1.136120  4.183113  2.180027
prestige_2  0.272168  0.942767  0.506548
prestige_3  0.133377  0.515419  0.262192
prestige_4  0.093329  0.479411  0.211525


#### 4.5 Interpret the OR of Prestige_2

Answer: Odds of being admitted decreased by 50% for students from prestige_2

#### 4.6 Interpret the OR of GPA

Answer: for each unit of positive change in GPA, there are double increase in odds of admission

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [27]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [28]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [29]:
combos.head()

,0,1,2,3
0,220.0,2.260000,1.0,1.0
1,220.0,2.260000,2.0,1.0
2,220.0,2.260000,3.0,1.0
3,220.0,2.260000,4.0,1.0
4,220.0,2.453333,1.0,1.0


In [30]:
# recreate the dummy variables
combos.columns = ['gre','gpa','prestige','intercept']
combos['prestige']=combos.prestige.astype(int)
dummy_ranks = pd.get_dummies(combos['prestige'],prefix='prestige')


In [31]:

# keep only what we need for making predictions
col_to_keep=['gre','gpa','prestige','intercept']
combos=combos[col_to_keep].join(dummy_ranks.ix[:,'prestige_2':])
combos.head()

,gre,gpa,prestige,intercept,prestige_2,prestige_3,prestige_4
0,220.0,2.260000,1,1.0,0,0,0
1,220.0,2.260000,2,1.0,1,0,0
2,220.0,2.260000,3,1.0,0,1,0
3,220.0,2.260000,4,1.0,0,0,1
4,220.0,2.453333,1,1.0,0,0,0


#### 5.2 Make predictions on the enumerated dataset

In [32]:
train_col= ['gre','gpa','prestige_2','prestige_3','prestige_4']
admit_pred=lm_f.predict(combos[train_col])
combos['admit_pred']=admit_pred

print combos.tail(4)

       gre  gpa  prestige  intercept  prestige_2  prestige_3  prestige_4  \
396  800.0  4.0         1        1.0           0           0           0   
397  800.0  4.0         2        1.0           1           0           0   
398  800.0  4.0         3        1.0           0           1           0   
399  800.0  4.0         4        1.0           0           0           1   

     admit_pred  
396    0.734040  
397    0.582995  
398    0.419833  
399    0.368608  


#### 5.3 Interpret findings for the last 4 observations

Answer: the last 4 observations have same gre and gpa data but different prestige which resulted in different admit prediction. Student from prestige 1 school has 73% probability of admission, while student from prestige 2, 3 and 4 have 58%, 42% and 37% probability of admission respectively

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.